In [ ]:
using Catalyst
using OrderedCollections
using Catalyst.Combinatorics
#this fucking works lfgggg

t = Catalyst.DEFAULT_IV
function funcsym(S::Symbol, t, args...)
    S = Symbol(S,args...)
    (@species $(S)(t))[1]
end
function gen_rates(n)
    ptoids = OrderedDict{Symbol,Int}()
    for i in 1:n
        psym = Symbol("k",i)
        ptoids[psym] = i
    end
    return [(@parameters $psym)[1] for psym in keys(ptoids)]
end
function gen_species(n)
    species=[]
    for i in 1:n
        push!(species, (funcsym(Symbol("X",i),t)))
    end
    for size in 2:n
        for combo in combinations(1:n, size)
            name = join(["X$i" for i in combo])
            push!(species, (funcsym(Symbol(name),t)))
        end
    end
    return species#[(@parameters $s)[1] for s in species]
end
specs = gen_species(4)
rates = gen_rates(4)
print(specs)
@parameters t
print(Reaction(rates[1],[specs[1],specs[2]], [specs[3]]))





Any[X1(t), X2(t), X3(t), X4(t), X1X2(t), X1X3(t), X1X4(t), X2X3(t), X2X4(t), X3X4(t), X1X2X3(t), X1X2X4(t), X1X3X4(t), X2X3X4(t), X1X2X3X4(t)]k1, X1 + X2 --> X3

In [ ]:




function generate_nmer_association(n)
    #@parameters t
    species_list = []
    reactions = []
    params = []
    
    # Start with monomer species X1 to Xn
    for i in 1:n
        push!(species_list, @species Symbol("X",i))
    end
    
    # Dictionary to keep track of complexes
    complexes = Dict{}()
    
    # Generate reactions for associations and dissociations
    for i in 2:n
        # Define the complex species X[i-1]X[i]
        @species Symbol("X$(i-1)_X$(i)(t)")
        complex_name = @species Symbol("X$(i-1)_X$(i)")
        complexes[(i-1, i)] = complex_name
        push!(species_list, complex_name)
        
        # Define reaction parameters
        k_on_sym = @parameters Symbol("k_on$(i)")
        k_off_sym = @parameters Symbol("k_off$(i)")
        push!(params, k_on_sym, k_off_sym)
        @species Symbol("X$(i-1)(t)")
        @species Symbol("X$(i)(t)")
        print(Reaction(k_on_sym, [(Symbol("X$(i-1)")), (Symbol("X$(i)"))], [complex_name]))
        # Association: X[i-1] + X[i] --> X[i-1]_X[i]
        push!(reactions, Reaction(k_on_sym, [(@species Symbol("X$(i-1)")), (@species Symbol("X$(i)"))], [complex_name]))
        
        # Dissociation: X[i-1]_X[i] --> X[i-1] + X[i]
        push!(reactions, Reaction(k_off_sym, [complex_name], [(@species Symbol("X$(i-1)")), (@species Symbol("X$(i)"))]))
    end
    println(reactions)
    # Optionally, you can extend this to higher-order complexes
    # For example, associate complexes with monomers to form larger complexes
    # Below is an example of extending to higher-order complexes
    for i in 3:n
        prev_complex = complexes[(i-2, i-1)]
        print("Hello")
        new_complex = @species Symbol("X$(i-2)_X$(i-1)_X$(i)")
        complexes[(i-2, i-1, i)] = new_complex
        push!(species_list, new_complex)
        
        # Parameters for higher-order association/dissociation
        k_on_sym = @species Symbol("k_on_complex$(i)")
        k_off_sym = @species Symbol("k_off_complex$(i)")
        push!(params, k_on_sym, k_off_sym)
        
        # Association: Previous Complex + X[i] --> New Complex
        push!(reactions, Reaction(k_on_sym, [prev_complex, (@species Symbol("X$(i)"))], [new_complex]))
        
        # Dissociation: New Complex --> Previous Complex + X[i]
        push!(reactions, Reaction(k_off_sym, [new_complex], [prev_complex, (@species Symbol("X$(i)"))]))

    end
    
    # Create the ReactionSystem
    rn = ReactionSystem(reactions, t, species_list, params)
    return rn
end

generate_nmer_association (generic function with 1 method)

In [52]:
generate_nmer_association(3)

ErrorException: Sym Symbol(X2(t)) is not callable. Use @syms Symbol(X2(t))(var1, var2,...) to create it as a callable.

In [54]:
print(Reaction(k_on_sym, [(@parameters Symbol("X$(i-1)")[1]), (@parameters Symbol("X$(i)")[1])], [@parameters Symbol("X$(i-1)_X$(i)")[1]]))

UndefVarError: UndefVarError: `k_on_sym` not defined

In [ ]:
using Catalyst
using DifferentialEquations
using ModelingToolkit



function get_fc_species_list(n::Int64)::Vector{Vector{Variable}}
    X = @species X[1:n] # Creates an array of Variable objects
    monomers = X
    num_subsets = 2^n - 1
    species_list = Vector{Vector{Variable}}(undef, num_subsets)
    for i in 1:num_subsets
        subset = Variable[]
        for j in 1:n
            if (i >> (j - 1)) & 1 == 1
                push!(subset, monomers[j])
            end
        end
        species_list[i] = subset
    end
    return species_list
end

function construct_reactions(n::Int64)
    species_list = get_fc_species_list(n)
    num_subsets = length(species_list)
    k = @parameters k[1:num_subsets, 1:num_subsets] # Define rate constants as Parameters
    rxs = Reaction[]
    for i in 1:num_subsets
        reactants = species_list[i]
        for j in 1:num_subsets
            products = species_list[j]
            if reactants != products
                rate_constant = k[i, j]
                push!(rxs, Reaction(rate_constant, reactants, products))
            end
        end
    end
    return rxs
end

function get_fc_reaction_network(n::Int64)
    t = @species t
    rxs = construct_reactions(n)
    rn = ReactionSystem(rxs, t)
    return rn
end